In [1]:
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F

## 5.1 Layers and Blocks

In [2]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
print(X)
net(X)

tensor([[0.1723, 0.4092, 0.1230, 0.2709, 0.7299, 0.4572, 0.4240, 0.3181, 0.2789,
         0.5940, 0.4275, 0.3392, 0.1520, 0.0078, 0.5674, 0.0342, 0.1380, 0.5279,
         0.7189, 0.8523],
        [0.0889, 0.5589, 0.4652, 0.7209, 0.4513, 0.9897, 0.4386, 0.2648, 0.1269,
         0.2176, 0.1141, 0.6791, 0.7231, 0.7229, 0.3700, 0.1421, 0.3846, 0.6273,
         0.3931, 0.5031]])


tensor([[ 0.1318,  0.0735, -0.0101, -0.0059, -0.0420,  0.0982,  0.0775,  0.1077,
         -0.0483,  0.1220],
        [ 0.1162,  0.0119,  0.0971, -0.1180, -0.0475,  0.0361,  0.0220,  0.0857,
         -0.0618,  0.0840]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
    '''
    Notice how we are using forward
    '''
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
mlp = MLP()
mlp(X)

tensor([[-0.1386,  0.0078, -0.0087, -0.0227,  0.0253,  0.1303, -0.2619, -0.0163,
          0.2790, -0.0940],
        [-0.1019,  0.0089,  0.0549, -0.0990, -0.0103,  0.1711, -0.1036, -0.0360,
          0.2559, -0.1930]], grad_fn=<AddmmBackward>)

In [5]:
class Mysequential(nn.Module):
    '''
    Implementation of sequential class
    '''
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
myseq = Mysequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
myseq(X)

tensor([[-0.0129, -0.0724, -0.0081,  0.1698,  0.0265, -0.1036, -0.0710, -0.1400,
          0.0370, -0.0807],
        [-0.0422, -0.1870,  0.0312,  0.2074, -0.0779, -0.1150, -0.0879, -0.0227,
          0.1130, -0.0098]], grad_fn=<AddmmBackward>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0505, grad_fn=<SumBackward0>)

## 5.2 Parameter Managemnt

In [8]:
X = torch.rand(2,4)
print(X)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
net(X)

tensor([[0.3698, 0.4003, 0.3178, 0.5835],
        [0.0183, 0.8520, 0.2973, 0.8018]])


tensor([[0.1210],
        [0.1227]], grad_fn=<AddmmBackward>)

In [9]:
print(net[2].state_dict())
print(net[1].state_dict())

OrderedDict([('weight', tensor([[-0.2078, -0.2519,  0.1759,  0.2456, -0.0524,  0.0837, -0.1210,  0.2925]])), ('bias', tensor([0.1129]))])
OrderedDict()


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1129], requires_grad=True)
tensor([0.1129])


In [11]:
net[2].weight.grad == None

True

In [12]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data

tensor([0.1129])

In [14]:
def block1():
    net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
    return net
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
rgnet = nn.Sequential(block2(),nn.Linear(4,1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0129, -0.0011, -0.0018, -0.0093]), tensor(0.))

In [16]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [17]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)== nn.Linear:
        nn.init.constant_(m.weight,42)
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0679, -0.1876,  0.1519,  0.5441])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [56]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, 5.6976, -0.0000, 6.7956],
        [8.5974, 9.0064, 0.0000, 7.9633]], grad_fn=<SliceBackward>)

In [55]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.5117,  1.0000,  1.0000])

In [57]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
